In [140]:
using Plots, Colors, DataFrames, CSV, GLM, RegressionTables

cd("/Users/junwong/Dropbox/Second Year/Dingel - Trade/Assignments")

# Quantitative Model

## Calibrating $X_{ni}$

In [141]:
df = DataFrame(CSV.File("data/bilateral_trade_country/bilateral_trade_country.csv"))

Xₙᵢ⁰ = unstack(df, :country_dest, :country_org, :trade_flow2014)
Xₙᵢ⁰ = Array(Xₙᵢ⁰[!,2:43])

exp⁰ = combine(groupby(df, :country_org), :trade_flow2014 => sum, renamecols=false);
imp⁰ = combine(groupby(df, :country_dest), :trade_flow2014 => sum, renamecols=false);
rename!(imp⁰, :trade_flow2014 => :imports)
yₙ = exp⁰[!,2] ./ imp⁰[!,2];

df = leftjoin(df, imp⁰, on=:country_dest);
transform!(df, [:trade_flow2014, :imports] => (./) => :pi_ni);
πₙᵢ = unstack(df, :country_dest, :country_org, :pi_ni);
πₙᵢ = Array(πₙᵢ[!, 2:43]);

## Impact of a 10% increase in Chinese productivity

In [142]:
# Setting T̂ₙᵢ
T̂ₙᵢ = ones(42,1);
T̂ₙᵢ[8] = 1.1;
θ = 4;

# Initialize vectors
ŵ = ones(42, 1);

# Transfer numeraire
h(w) = w[end]

# Parameters
tol = 1e-10
κ=0.01

0.01

In [143]:
# Excess demand function 
excess_demand = function(ŵ, Xₙᵢ⁰, T̂, πₙᵢ, yₙ, θ, w0)
    # get the denominator that is by n
    dni = zeros(length(T̂), length(T̂))  
    for i = 1:size(Xₙᵢ⁰, 2)
        for n = 1:size(Xₙᵢ⁰, 1)
            dni[n, i] = πₙᵢ[n, i] .* T̂[i] .* (ŵ[i] .^ (-θ))
        end
    end 
    
    # get the rest of the calculations
    res_i = zeros(length(T̂), 1)
    for i = 1:size(Xₙᵢ⁰, 2)
        res_by_n = zeros(length(T̂), 1)
        denom_i = sum(Xₙᵢ⁰[:,i])
        for n = 1:size(Xₙᵢ⁰, 1)
            num_ni = Xₙᵢ⁰[n, i] * T̂[i] * ((ŵ[i]) ^ (-θ))
            num_n = yₙ[n] .* ŵ[n] .+ (1 .- yₙ[n]) .* h(w0 .* ŵ) ./ h(w0)
            denom_n = sum(dni[n, :])
            res_by_n[n] = num_n * num_ni / denom_i / denom_n
        end
        res_i[i] = sum(res_by_n) - ŵ[i]
    end
    
    return res_i
end 

#37 (generic function with 1 method)

In [144]:
# Iterating wrapper
iterating_fn = function(ŵ, Xₙᵢ⁰, T̂, πₙᵢ, yₙ, θ, w0, tol, κ) 
    zᵢ = ones(length(T̂))
    
    while maximum(abs.(zᵢ)) > tol
        #iter +=1
        zᵢ = excess_demand(ŵ, Xₙᵢ⁰, T̂, πₙᵢ, yₙ, θ, w0) 
        ŵ = ŵ .+ κ .* zᵢ
        ŵ[end] = 1
    end 
    
    return [ŵ, zᵢ]
end     

#39 (generic function with 1 method)

In [145]:
# Run function
x = iterating_fn(ones(42, 1), Xₙᵢ⁰, T̂ₙᵢ, πₙᵢ, yₙ, θ, ones(42,1), tol, κ)

2-element Vector{Matrix{Float64}}:
 [1.0003551063453036; 1.0001388093848207; … ; 1.0005777579017796; 1.0]
 [2.5399460312769406e-11; 2.4520385721871207e-11; … ; 2.3367086043890595e-11; -9.988654348092041e-11]

## Changes in Real Wage and Expenditures

Construct terms

In [146]:
Φ̂ₙ = zeros(42)
ni = zeros(42, 42)
for n=1:42
    for i=1:42
        ni[n, i] = T̂ₙᵢ[n] * πₙᵢ[n,i] * (x[1][i]^(-θ)) # add T in here 
    end
    Φ̂ₙ[n] = sum(ni[n, :])
end

# New CES price index
P̂ₙ = (Φ̂ₙ).^(-1/θ);

# Real wage
ω̂ = x[1] ./ P̂ₙ

# Share of spending of n on i 
π̂ₙᵢ = zeros(42, 42)
dₙᵢ = ones(42, 42)
for n = 1:size(dₙᵢ, 1)
    for i = 1:size(dₙᵢ, 2) 
        π̂ₙᵢ[n, i] = T̂ₙᵢ[i] * (dₙᵢ[n, i] * x[1][i])^(-θ) / Φ̂ₙ[n]
    end
end

# Change in domestic trade share 
π̂ₙₙ = T̂ₙᵢ .* (x[1] ./ P̂ₙ).^(-θ)

# New expenditures
X̂ₙ = x[1] #.+ h(x[1])
Χ̂ₙ = X̂ₙ ./ P̂ₙ;

In [147]:
# Plot results by country
cntry = CSV.File("data/bilateral_trade_country/country_list.csv") |> Tables.matrix
bar(cntry, (ω̂ .- 1)* 100, ylabel="Percent change in real wages", legend=false)
savefig("output/q1_tfp_shock_real_wages.pdf")
bar(cntry, (Χ̂ₙ .- 1) * 100, ylabel = "Percent change in real expenditures", legend=false)
savefig("output/q1_tfp_shock_real_exp.pdf")

Regressions

In [148]:
reg_data = hcat(log.(ω̂), log.(Χ̂ₙ), log.(Φ̂ₙ),  log.(π̂ₙₙ), πₙᵢ[:,8])
reg_data = DataFrame(reg_data, :auto)
rename!(reg_data, [:real_wage, :real_exp, :market_acc, :domestic_share, :initial_chn]);
dict = Dict("real_wage"=> "Real Wages", "real_exp" => "Real Expenditures", "market_acc" => "Market Access", "domestic_share" => "Domestic Share", "initial_chn" => "Initial CHN Share");


In [149]:
r1 = lm(@formula(real_wage ~ market_acc + domestic_share + initial_chn), reg_data)
r2 = lm(@formula(real_exp ~ market_acc + domestic_share + initial_chn), reg_data)
regtable(r1,r2; renderSettings = latexOutput("output/q1_regression.tex"), 
        labels = dict, print_estimator_section = false)


## Changing numeraire

In [137]:
# Transfer numeraire
h(w) = w[8] # china is now transfer numeraire 
x = iterating_fn(ones(42, 1), Xₙᵢ⁰, T̂ₙᵢ, πₙᵢ, yₙ, θ, ones(42,1), tol, κ)

2-element Vector{Matrix{Float64}}:
 [0.9998698058580759; 0.998996848794946; … ; 0.9996290712580896; 1.0]
 [2.5781043966333073e-11; 2.388322872803883e-11; … ; 2.300626356088742e-11; -9.995781979910134e-11]

In [138]:
Φ̂ₙ = zeros(42)
ni = zeros(42, 42)
for n=1:42
    for i=1:42
        ni[n, i] = T̂ₙᵢ[n] * πₙᵢ[n,i] * (x[1][i]^(-θ))
    end
    Φ̂ₙ[n] = sum(ni[n, :])
end

# New CES price index
P̂ₙ = (Φ̂ₙ).^(-1/θ);

# Real wage
ω̂ = x[1] ./ P̂ₙ

# Share of spending of n on i 
π̂ₙᵢ = zeros(42, 42)
dₙᵢ = ones(42, 42)
for n = 1:size(dₙᵢ, 1)
    for i = 1:size(dₙᵢ, 2) 
        π̂ₙᵢ[n, i] = T̂ₙᵢ[i] * (dₙᵢ[n, i] * x[1][i])^(-θ) / Φ̂ₙ[n]
    end
end

# Change in domestic trade share 
π̂ₙₙ = T̂ₙᵢ .* (x[1] ./ P̂ₙ).^(-θ)

# New expenditures
X̂ₙ = x[1] #.+ h(x[1]);
Χ̂ₙ = X̂ₙ ./ P̂ₙ ;

In [139]:
bar(cntry, (ω̂ .- 1)* 100, ylabel="Percent change in real wages", legend=false)
savefig("output/q1_tfp_shock_chn_num_real_wages.pdf")
bar(cntry, (Χ̂ₙ .- 1) * 100, ylabel = "Percent change in real expenditures", legend=false)
savefig("output/q1_tfp_shock_chn_num_real_exp.pdf")